# Soil geometry

This notebook visualizes soil geometries and saves their figures. Soil geometry is expressed by a matrix of diffusion coefficients corresponding to soil types. The matrix is created by interpolating a seed soil matrix, which is a 2D array with a minimum 5 x 5 size. For our simulation, three types of soils are used. So, three diffusion coefficients are defined and are used to create a seed soil matrix.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from modules.classes import Quantity2D, Analytic, Interpolate
from modules.integrator import forward_euler_final
from modules.geometries import *
from modules.tests import plot_mass_conservation, test_gaussian

## Common setup

The stability of setting values should be checked with $\frac{(D \times dt)}{(dx)^2}$ < 0.5.

In [ ]:
#### common setup

n_grid = 100
n_time = 500
dt = 0.0001
dx = 0.01
dy = 0.01

# Diffusion coefficients of soils m^2 s^-1
D_high = 1.3e-12
D_midium = 1.2e-12
D_low = 1.1e-12
D_background = 1.5e-12

conc = Quantity2D(
    n_grid,
    n_time,
    (-0.5*n_grid*dx, 0.5*n_grid*dx),
    (-0.5*n_grid*dy, 0.5*n_grid*dy),
    (0, dt*n_time),
)

# array of actual x and y values on grid points
xcoords = conc.xcoords
ycoords = conc.ycoords

X, Y = np.meshgrid(xcoords, ycoords)

# create a 100 x 100 grid to hold factors for a gausian distribution
initial_condition = 1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)
sources =  1*np.exp(- (X**2 + Y**2)/(0.05*dx*n_grid)**2)


## Soil Geometries

In [ ]:
fig_t2, ax_t2 = plot_soil_geometry(conc, triangle_2, [D_high, D_midium], D_background)
fig_t2.savefig(f'./plots/soil_matrix_{triangle_2.__name__}.png')


In [ ]:
fig_sqv2, ax_sqv2 = plot_soil_geometry(conc, square_vertical_2layers, [D_high, D_midium], D_background)
fig_sqv2.savefig(f'./plots/soil_matrix_{square_vertical_2layers.__name__}.png')


In [ ]:
fig_s, ax_s = plot_soil_geometry(conc, square_layers, [D_high, D_midium, D_low], D_background)
fig_s.savefig(f'./plots/soil_matrix_{square_layers.__name__}.png')
